In [1]:
#imports and user defined errors 
import re
import copy
import random

class GateNotDefined(Exception):
    '''
    Error to be raised when the gate is not found in the the basic gates 
    '''
    def __init__(self, gate):
        self.message = f"{gate} not defined"
        super().__init__(self.message)

class GateReprError(Exception):
    '''
    Error to be raised for illegal gate declaration format 
    '''
    def __init__(self, gate_repr):
        self.message = f"{gate_repr} not valid"
        super().__init__(self.message)

#custom errors to be used
class FloatingOutput(Exception):
    '''
    Error to be raised when an output of a gate is floating
    '''
    def __init__(self, node_name):
        self.message = f"{node_name} is floating, no driver for output node is defined"
        super().__init__(self.message)

class CirNotLevelized(Exception):
    '''
    Error to be raised if a function that requires levelized circuit is called
    before the circuit is levelized
    '''
    def __init__(self):
        self.message = f"Circuit is not levelized! Levelize circuit and try again"
        super().__init__(self.message)

class InputUndefined(Exception):
    '''
    This error is raised when a circuit is tried to simulated without input assignment
    '''
    def __init__(self, node):
        self.message = f"{node.name} not assigned an input value"
        super().__init__(self.message)

class FaultBeforeFaultsite(Exception):
    '''
    This error is raised when a fault is detected before the fault site
    '''
    def __init__(self, fault, output):
        self.message = f"Fault value of {output} observed before fault site {fault}"
        super().__init__(self.message)

class FaultReprError(Exception):
    '''
    This error is raised when the fault input from the user is not of the expected format
    '''
    def __init__(self, fault_input = None, fault_output = None, fault_type = None):
        self.message = "Fault representaiton Error!"
        if fault_input != None:
            self.message = self.message + f"{fault_input} not a node in the circuit"
        elif fault_output != None:
            self.message = self.message + f"{fault_output} not a node in the circuit"
        elif fault_type != None:
            self.message = self.message + f"sa{fault_type} not a valid fault"

class ParentNodeControllabilityNA(Exception):
    def __init__(self, node_name):
        self.message = f"Node {node_name}'s controllability not available. Levelize circuit before doing scoap analysis"
        super().__init__(self.message)


In [2]:
#basic classes used for circuit representation
class node_type:
    input_node = 0
    output_node = 1
    internal_wire = 3

#class to represnt the value of each node
class node_value:
    zero = 0
    one = 1
    d = 2
    d_bar = 3
    undefined = -1
    str_repr = {zero: "0", one: "1", d: "D", d_bar:"D'", undefined: "X"}

class gate_type:
    AND_gate = 0
    OR_gate = 1
    NOT_gate = 2
    NAND_gate = 3
    NOR_gate = 4
    XOR_gate = 5
    XNOR_gate = 6
    BUFF_gate = 7

    @staticmethod
    def AND(input_list, output_node):
        output = input_list[0].get_value(output_node)

        for i in range(1, len(input_list)):
            if output == node_value.zero or output == node_value.undefined:
                return output
            if input_list[i].get_value(output_node) == node_value.zero:
                output =  node_value.zero
            elif input_list[i].get_value(output_node) == node_value.undefined:
                output = node_value.undefined
            elif input_list[i].get_value(output_node) == node_value.one:
                continue
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.d_bar:
                    output =  node_value.zero
                else:
                    output = node_value.d
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.d:
                    output =  node_value.zero
                else:
                    output = node_value.d_bar

        return output
    
    @staticmethod
    def OR(input_list, output_node):
        output = input_list[0].get_value(output_node)

        for i in range(1, len(input_list)):
            if output == node_value.one or output == node_value.undefined:
                return output
            if input_list[i].get_value(output_node) == node_value.one:
                output =  node_value.one
            elif input_list[i].get_value(output_node) == node_value.undefined:
                output =  node_value.undefined
            elif input_list[i].get_value(output_node) == node_value.zero:
                continue
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.d_bar:
                    output = node_value.one
                else:
                    output = node_value.d
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.d:
                    output = node_value.one
                else:
                    output = node_value.d_bar

        return output
    
    @staticmethod
    def NOT(input_list, output_node):
        output_dict = {node_value.zero: node_value.one,
                       node_value.one: node_value.zero,
                       node_value.d: node_value.d_bar,
                       node_value.d_bar: node_value.d,
                       node_value.undefined: node_value.undefined}
        
        return output_dict[input_list[0].get_value(output_node)]
    
    @staticmethod
    def BUFF(input_list, output_node):
        
        return input_list[0].get_value(output_node)
    
    @staticmethod
    def NAND(input_list, output_node):
        inv_output = {node_value.zero: node_value.one,
                       node_value.one: node_value.zero,
                       node_value.d: node_value.d_bar,
                       node_value.d_bar: node_value.d,
                       node_value.undefined: node_value.undefined}
        
        output = input_list[0].get_value(output_node)

        for i in range(1, len(input_list)):
            if output == node_value.zero or output == node_value.undefined:
                return inv_output[output]
            if input_list[i].get_value(output_node) == node_value.zero:
                output =  node_value.zero
            elif input_list[i].get_value(output_node) == node_value.undefined:
                output = node_value.undefined
            elif input_list[i].get_value(output_node) == node_value.one:
                continue
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.d_bar:
                    output =  node_value.zero
                else:
                    output = node_value.d
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.d:
                    output =  node_value.zero
                else:
                    output = node_value.d_bar

        return inv_output[output]
    
    @staticmethod
    def NOR(input_list, output_node):
        inv_output = {node_value.zero: node_value.one,
                       node_value.one: node_value.zero,
                       node_value.d: node_value.d_bar,
                       node_value.d_bar: node_value.d,
                       node_value.undefined: node_value.undefined}
        
        output = input_list[0].get_value(output_node)

        for i in range(1, len(input_list)):
            if output == node_value.one or output == node_value.undefined:
                return inv_output[output]
            if input_list[i].get_value(output_node) == node_value.one:
                output =  node_value.one
            elif input_list[i].get_value(output_node) == node_value.undefined:
                output =  node_value.undefined
            elif input_list[i].get_value(output_node) == node_value.zero:
                continue
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.d_bar:
                    output = node_value.one
                else:
                    output = node_value.d
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.d:
                    output = node_value.one
                else:
                    output = node_value.d_bar

        return inv_output[output]
    
    @staticmethod
    def XOR(input_list, output_node):
        inverted_output = {node_value.zero: node_value.one,
                          node_value.one: node_value.zero,
                          node_value.d: node_value.d_bar,
                          node_value.d_bar: node_value.d,
                          node_value.undefined: node_value.undefined}
        
        output = None
        if input_list[0].get_value(output_node) == node_value.undefined:
            return node_value.undefined
        elif input_list[0].get_value(output_node) == node_value.zero:
            output = input_list[1].get_value(output_node)
        elif input_list[0].get_value(output_node) == node_value.one:
            output = inverted_output[input_list[1].get_value(output_node)]
        elif input_list[0].get_value(output_node) == node_value.d:
            if input_list[1].get_value(output_node) == node_value.d_bar:
                output = node_value.one
            elif input_list[1].get_value(output_node) == node_value.one:
                output = inverted_output[node_value.d]
            elif input_list[1].get_value(output_node) == node_value.zero:
                output = node_value.d
            else:
                output = node_value.zero
        elif input_list[0].get_value(output_node) == node_value.d_bar:
            if input_list[1].get_value(output_node) == node_value.d:
                output = node_value.one
            elif input_list[1].get_value(output_node) == node_value.one:
                output = inverted_output[node_value.d_bar]
            elif input_list[1].get_value(output_node) == node_value.zero:
                output = node_value.d_bar
            else:
                output = node_value.zero

        for i in range(2, len(input_list)):
            if input_list[i].get_value(output_node) == node_value.zero:
                continue
            elif input_list[i].get_value(output_node) == node_value.one:
                output = inverted_output[output]
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.zero:
                    output = node_value.d
                elif output == node_value.one:
                    output = inverted_output[node_value.d]
                elif output == node_value.d:
                    output = node_value.zero
                elif output == node_value.d_bar:
                    output = node_value.one
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.zero:
                    output = node_value.d_bar
                elif output == node_value.one:
                    output = inverted_output[node_value.d_bar]
                elif output == node_value.d:
                    output = node_value.one
                elif output == node_value.d_bar:
                    output = node_value.zero
    
        return output
    
    @staticmethod
    def XNOR(input_list, output_node):
        inverted_output = {node_value.zero: node_value.one,
                          node_value.one: node_value.zero,
                          node_value.d: node_value.d_bar,
                          node_value.d_bar: node_value.d,
                          node_value.undefined: node_value.undefined}
        
        output = None
        if input_list[0].get_value(output_node) == node_value.undefined:
            return node_value.undefined
        elif input_list[0].get_value(output_node) == node_value.one:
            output = input_list[1].get_value(output_node)
        elif input_list[0].get_value(output_node) == node_value.zero:
            output = inverted_output[input_list[1].get_value(output_node)]
        elif input_list[0].get_value(output_node) == node_value.d:
            if input_list[1].get_value(output_node) == node_value.d_bar:
                output = node_value.zero
            elif input_list[1].get_value(output_node) == node_value.zero:
                output = inverted_output[node_value.d]
            elif input_list[1].get_value(output_node) == node_value.one:
                output = node_value.d
            else:
                output = node_value.one
        elif input_list[0].get_value(output_node) == node_value.d_bar:
            if input_list[1].get_value(output_node) == node_value.d:
                output = node_value.zero
            elif input_list[1].get_value(output_node) == node_value.zero:
                output = inverted_output[node_value.d_bar]
            elif input_list[1].get_value(output_node) == node_value.one:
                output = node_value.d_bar
            else:
                output = node_value.one

        for i in range(2, len(input_list)):
            if input_list[i].get_value(output_node) == node_value.one:
                continue
            elif input_list[i].get_value(output_node) == node_value.zero:
                output = inverted_output[output]
            elif input_list[i].get_value(output_node) == node_value.d:
                if output == node_value.one:
                    output = node_value.d
                elif output == node_value.zero:
                    output = inverted_output[node_value.d]
                elif output == node_value.d:
                    output = node_value.one
                elif output == node_value.d_bar:
                    output = node_value.zero
            elif input_list[i].get_value(output_node) == node_value.d_bar:
                if output == node_value.one:
                    output = node_value.d_bar
                elif output == node_value.zero:
                    output = inverted_output[node_value.d_bar]
                elif output == node_value.d_bar:
                    output = node_value.one
                elif output == node_value.d:
                    output = node_value.zero

        return output
                
class fault_types:
    sa0 = 0
    sa1 = 1

In [3]:
#class to represent a fault
class fault:
    '''
    This class represents a fault.

    Attributes:
    fault_type: Type of fault, either stuck at 0 or stuck at 1
    fault_node: Node to which the fault is associated with
    fault_output: Output linked with the node where the fault is associated
    '''

    def __init__(self, fault_type, fault_node, fault_output = None):
        self.fault_type = fault_type
        self.fault_node = fault_node
        self.fault_output = fault_output
        self.fault_string = None
        if self.fault_output == None:
            if self.fault_type == fault_types.sa0:
                self.fault_string = f"{self.fault_node}-0"
            else:
                self.fault_string = f"{self.fault_node}-1"
        else:
            if self.fault_type == fault_types.sa0:
                self.fault_string = f"{self.fault_output}-{self.fault_node}-0"
            else:
                self.fault_string = f"{self.fault_output}-{self.fault_node}-1"

    def __repr__(self):
        '''
        Returns the string representation of a fault
        '''
        return self.fault_string

In [4]:
#class to represent a gate
class gate:
    '''
    This class is used to represent a gate

    Attributes:
    num_inputs: Number of inputs(int)
    type: Type of the gate(gate_type)
    input_nodes: Nodes that are fed in
    Output node: Output node of the gate
    '''

    def __init__(self, string_representation):
        '''
        This function creates a gate

        param[in] string_representation String representation of the gate
                                        The string after = in the circuit bench file
        '''
        self.input_nodes = []
        self.output_node = None
        self.type = None
        self.num_inputs = 0
        self.gate_string = ""
        self.gate_function = None

        self.__update_gate_type(string_representation)

    def __update_gate_type(self, string_representation: str):
        '''
        This function updates the gate type
        '''
        gate_dict = {"AND" : [gate_type.AND_gate, gate_type.AND],
                     "OR"  : [gate_type.OR_gate, gate_type.OR],
                     "NOT" : [gate_type.NOT_gate, gate_type.NOT],
                     "NOR" : [gate_type.NOR_gate, gate_type.NOR],
                     "NAND": [gate_type.NAND_gate, gate_type.NAND],
                     "XOR" : [gate_type.XOR_gate, gate_type.XOR],
                     "XNOR": [gate_type.XNOR_gate, gate_type.XNOR],
                     "BUFF": [gate_type.BUFF_gate, gate_type.BUFF]}

        self.gate_string = re.match(r'^[^()]+', string_representation)

        if self.gate_string:
            self.gate_string = self.gate_string.group()
        else:
            raise GateReprError(string_representation)
        

        if self.gate_string not in gate_dict:
            raise GateNotDefined(self.gate_string)
        else:
            self.type = gate_dict[self.gate_string][0]
            self.gate_function = gate_dict[self.gate_string][1]
            self.input_nodes = re.findall(r'\((.*?)\)', string_representation)[0]
            self.input_nodes = self.input_nodes.split(",")
            for i in range(len(self.input_nodes)):
                self.input_nodes[i] = self.input_nodes[i].lstrip()
            self.num_inputs = len(self.input_nodes)


    def get_output(self):
        self.output_node.value = self.gate_function(self.input_nodes, self.output_node)

    def __repr__(self):
        '''
        Return a string representing the gate information
        '''
        str_repr = f"{self.num_inputs}-input {self.gate_string} gate | Input nodes: "
        for node in self.input_nodes:
            str_repr = str_repr + f"{node},"
        str_repr = str_repr[:-1] + " | "
        str_repr = f"{str_repr}Output node: {self.output_node}"
        return str_repr

In [5]:
class Controllability:
    '''
    This class is used to do the scoap analysis and get the c0 and c1 of each node

    Attributes
    c0: Controllability of 0
    c1: Controllability of 1
    '''
    def __init__(self, n, node_gate: gate):
        self.c0 = None
        self.c1 = None
        if n.type == node_type.input_node:
            self.c0 = 1
            self.c1 = 1
        else:
            self.__get_controllability(node_gate)

    def __get_controllability(self, node_gate: gate):
        if node_gate.output_node.type == node_type.input_node:
            self.c0 = 1
            self.c1 = 1
            return
        else:
            for n in node_gate.input_nodes:
                if n.controllability.c0 == None or n.controllability.c1 == None:
                    raise ParentNodeControllabilityNA(n.name)
                
            if node_gate.type == gate_type.BUFF_gate:
                self.__buff_controllability(node_gate)
            elif node_gate.type == gate_type.NOT_gate:
                self.__not_controllability(node_gate)
            elif node_gate.type == gate_type.AND_gate:
                self.__and_controllability(node_gate)
            elif node_gate.type == gate_type.OR_gate:
                self.__or_controllability(node_gate)
            elif node_gate.type == gate_type.NAND_gate:
                self.__nand_controllability(node_gate)
            elif node_gate.type == gate_type.NOR_gate:
                self.__nor_controllability(node_gate)
            elif node_gate.type == gate_type.XOR_gate:
                self.__xor_controllability(node_gate)
            elif node_gate.type == gate_type.XNOR_gate:
                self.__xnor_controllability(node_gate)

    def __buff_controllability(self, node_gate: gate):
        self.c0 == node_gate.input_nodes[0].controllability.c0 + 1
        self.c1 == node_gate.input_nodes[0].controllability.c1 + 1

    def __not_controllability(self, node_gate: gate):
        self.c0 = node_gate.input_nodes[0].controllability.c1 + 1
        self.c1 = node_gate.input_nodes[0].controllability.c0 + 1

    def __and_controllability(self, node_gate: gate):
        input_c0_list = [n.controllability.c0 for n in node_gate.input_nodes]
        input_c1_list = [n.controllability.c1 for n in node_gate.input_nodes]

        self.c0 = min(input_c0_list) + 1
        self.c1 = sum(input_c1_list) + 1

    def __or_controllability(self, node_gate: gate):
        input_c0_list = [n.controllability.c0 for n in node_gate.input_nodes]
        input_c1_list = [n.controllability.c1 for n in node_gate.input_nodes]

        self.c0 = sum(input_c0_list) + 1
        self.c1 = min(input_c1_list) + 1

    def __nand_controllability(self, node_gate: gate):
        input_c0_list = [n.controllability.c0 for n in node_gate.input_nodes]
        input_c1_list = [n.controllability.c1 for n in node_gate.input_nodes]

        self.c1 = min(input_c0_list) + 1
        self.c0 = sum(input_c1_list) + 1

    def __nor_controllability(self, node_gate: gate):
        input_c0_list = [n.controllability.c0 for n in node_gate.input_nodes]
        input_c1_list = [n.controllability.c1 for n in node_gate.input_nodes]

        self.c0 = min(input_c1_list) + 1
        self.c1 = sum(input_c0_list) + 1

    def __xor_controllability(self, node_gate: gate):
        c0_sum = node_gate.input_nodes[0].controllability.c0 + \
                    node_gate.input_nodes[1].controllability.c0
        c1_sum = node_gate.input_nodes[0].controllability.c1 + \
                    node_gate.input_nodes[1].controllability.c1
        c0_c1_sum = node_gate.input_nodes[0].controllability.c0 + \
                    node_gate.input_nodes[1].controllability.c1
        c1_c0_sum = node_gate.input_nodes[0].controllability.c1 + \
                    node_gate.input_nodes[1].controllability.c0

        self.c0 = min([c0_sum, c1_sum])
        self.c1 = min([c0_c1_sum, c1_c0_sum]) 

    def __xnor_controllability(self, node_gate: gate):
        c0_sum = node_gate.input_nodes[0].controllability.c0 + \
                    node_gate.input_nodes[1].controllability.c0
        c1_sum = node_gate.input_nodes[0].controllability.c1 + \
                    node_gate.input_nodes[1].controllability.c1
        c0_c1_sum = node_gate.input_nodes[0].controllability.c0 + \
                    node_gate.input_nodes[1].controllability.c1
        c1_c0_sum = node_gate.input_nodes[0].controllability.c1 + \
                    node_gate.input_nodes[1].controllability.c0

        self.c1 = min([c0_sum, c1_sum])
        self.c0 = min([c0_c1_sum, c1_c0_sum]) 
    

In [6]:
#class to represent a node
class node:
    '''
    This class represents a node in the circuit

    Attributes:
    name: Name of the node(string)
    type: Type of the node(node_type)
    value: Value of the node(node_value)
    gate_type: Type of gate(String)
    nodes_fed_in: List of nodes fed in(node)
    fault_list: List of faults associated with this node
    '''

    def __init__(self, circuit_bench_line: str):
        '''
        Initialize the object. 

        param[in] circuit_bench_line: A line in the circuit bench file
        '''
        self.name = None
        self.type = None
        self._value = node_value.undefined
        self.gate = None
        self.level = None
        self.fault_list = {fault_types.sa0: [], fault_types.sa1: []}
        self.selected_fault = None
        self.controllability = None
        self.zero_count = 0
        self.one_count = 0

        #check if it is an input node
        if ("INPUT" in circuit_bench_line.upper()):
            self.type = node_type.input_node
            self.name = re.findall(r'\((.*?)\)', circuit_bench_line)[0]
            self.level = 0
        elif ("OUTPUT" in circuit_bench_line.upper()):
            self.type = node_type.output_node
            self.name = re.findall(r'\((.*?)\)', circuit_bench_line)[0]
        else:
            #its an internal wire
            self.type = node_type.internal_wire
            self.name = circuit_bench_line.split("=")[0] #split at "=" symbol and the first element in the list
                                                         #is the node name
            self.name = self.name.rstrip() #remove any trailing white spaces
            self.__update_gate(circuit_bench_line)

    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, new_value):
        if (self.selected_fault == None) or \
            (isinstance(self.selected_fault, fault) and \
            (self.selected_fault.fault_output != None)):
            self._value = new_value
        else:
            if new_value in [node_value.d, node_value.d_bar]:
                #faulty value before fault site is not possible
                #hence raise an error
                raise FaultBeforeFaultsite(self.selected_fault, node_value.str_repr[new_value])
            else:
                if self.selected_fault.fault_type == fault_types.sa0:
                    if new_value == node_value.one:
                        new_value = node_value.d
                else:
                    if new_value == node_value.zero:
                        new_value = node_value.d_bar
                self._value = new_value
        
        if self._value == node_value.zero:
            self.zero_count += 1
        if self.value == node_value.one:
            self.one_count += 1

    def get_value(self, output_node):
        '''
        This function is used to get the value of the node based on the
        node fed out

        output_node: node fed out
        '''
        return_value = self._value
        if (self.selected_fault != None) and \
           (isinstance(self.selected_fault, fault) and \
            (((isinstance(self.selected_fault.fault_output, str)) and \
              (self.selected_fault.fault_output == "out")) or \
             ((isinstance(self.selected_fault.fault_output, node)) and \
              (self.selected_fault.fault_output.name == output_node.name)) or \
             ((self.selected_fault.fault_output == output_node)))):
            if self._value in [node_value.d, node_value.d_bar]:
                #faulty value before fault site is not possible
                #hence raise an error
                raise FaultBeforeFaultsite(self.selected_fault, node_value.str_repr[self._value])
            if self.selected_fault.fault_type == fault_types.sa0:
                #sa0 fault, return d if the node value is one
                if self._value == node_value.one:
                    return_value = node_value.d
            else:
                #sa1 fault, return d' if the node value is zero
                if self._value == node_value.zero:
                    return_value = node_value.d_bar

        return return_value

    
    def __update_gate(self, circuit_bench_line):
        '''
        Updates the gate information of a node
        '''
        gate_string_representation = circuit_bench_line.split("=")[1]
        gate_string_representation = gate_string_representation.lstrip()
        self.gate = gate(gate_string_representation)
        self.gate.output_node = self

    def update(self, circuit_bench_line):
        '''
        This function updates the nodes properties based on the new circuit bench file
        '''
        if "OUTPUT" in circuit_bench_line.upper():
            #node is an output node
            self.type = node_type.output_node
        else:
            #an assignment done to the node, update the gate type
            self.__update_gate(circuit_bench_line)

    def find_level(self):
        if self.level != None:
            #level already found
            return
        
        prev_level = 0

        for input_node in self.gate.input_nodes:
            if input_node.level == None:
                input_node.find_level()

            if input_node.level > prev_level:
                prev_level = input_node.level   

        self.level = prev_level + 1

    def get_controllability(self):
        self.controllability = Controllability(self, self.gate)

    def create_fault_list(self, output_node = None):
        if output_node == None:
            if len(self.fault_list[fault_types.sa0]) == 0:
                self.fault_list[fault_types.sa0].append(fault(fault_types.sa0, self)) #creates a-0 fault
                if self.type == node_type.output_node:
                    self.fault_list[fault_types.sa0].append(fault(fault_types.sa0, self, "out"))
            if len(self.fault_list[fault_types.sa1]) == 0:
                self.fault_list[fault_types.sa1].append(fault(fault_types.sa1, self)) #creates a-1 fault
                if self.type == node_type.output_node:
                    self.fault_list[fault_types.sa1].append(fault(fault_types.sa1, self, "out"))
        else:
            self.fault_list[fault_types.sa0].append(fault(fault_types.sa0, self, output_node))
            self.fault_list[fault_types.sa1].append(fault(fault_types.sa1, self, output_node))
            return
        
        if self.gate != None:
            for input_node in self.gate.input_nodes:
                input_node.create_fault_list(self) #creates g-a-0 and g-a-1

    
    def select_fault(self, fault, output = None):
        '''
        Selects the fault based on the inputs given

        fault: Type of fault to select - sa0 or sa1
        Output: Output node associated with the fault
        '''

        if (isinstance(output, node) or (output == None)):
            for f in self.fault_list[fault]:
                if (isinstance(f.fault_output, node)):
                    if f.fault_output.name == output.name:
                        self.selected_fault = f
                        break
                elif ((f.fault_output == None)):
                    if f.fault_output == output:
                        self.selected_fault = f
                        break
        else:
            for f in self.fault_list[fault]:
                if (isinstance(f.fault_output, str)):
                    if f.fault_output == output:
                        self.selected_fault = f
                        break
        
    def __repr__(self):
        return self.name

In [7]:
#class to represent a circuit
class circuit:
    '''
    This class represents a circuit described in the circuit bench file

    Attributes:
    nodes: List of all the nodes(nodes)
    input_list: Index of input nodes in nodes list(int)
    output_list: Index of output nodes in nodes list(int)    
    '''
    
    def __init__(self, circuit_bench_file):
        '''
        Initializes the class

        param[in] circuit_bench_file: Path to the circuit bench file
        '''
        self.nodes = []
        self.input_list = []
        self.output_list = []
        self.internal_nodes = []
        self.node_index = {} #index of each node in the nodes list, key values for this dictionary are the node names
        self.levelized_nodes = None
        self.__fault_list_created = False
        self.num_levels = -1

        lines = self.__parse_circuit_bench_file(circuit_bench_file)
        #create nodes
        self.__create_nodes(lines)
        #check if all outputs are defined
        self.__check_output_definition()

    def __parse_circuit_bench_file(self, circuit_bench_file):
        '''
        This function parses the circuit bench file and sends out a list
        '''
        lines = []

        with open(circuit_bench_file, "r") as f:
            for line in f:
                line = line.replace("\n", "")
                if len(line) == 0:
                    continue
                lines.append(line)

        return lines
    
    def __create_nodes(self, lines):
        '''
        This function creates nodes from the lines read from the circuit bench file
        '''
        for line in lines:
            #check for comments
            if (line[0] == "#"):
                continue
            
            #check if node is already created - Possible when output is declared after using a wire
            node_name = re.findall(r'\((.*?)\)', line)[0]

            if "=" in line:
                node_name = line.split("=")[0]
                node_name = node_name.rstrip()

            if node_name in self.node_index:
                #update the node properties
                self.nodes[self.node_index[node_name]].update(line)
            else:
                #new node in the circuit
                #create a node and add it to the node list
                self.nodes.append(node(line))
                self.node_index[self.nodes[-1].name] = len(self.nodes) - 1
            

        #all the nodes are defined, now update the input nodes in the gates list to nodes
        #before this point, the input nodes in gate contains the node name alone and not the
        #node object
        for n in self.nodes:
            if n.gate != None:
                nodes_fed_in = n.gate.input_nodes
                for i in range(len(nodes_fed_in)):
                    nodes_fed_in[i] = self.nodes[self.node_index[nodes_fed_in[i]]]
                n.gate.input_nodes = nodes_fed_in

            #add nodes to the intern_node list
            if n.type == node_type.input_node:
                self.input_list.append(n)
            elif n.type == node_type.output_node:
                self.output_list.append(n)
            else:
                self.internal_nodes.append(n)

    
    def __check_output_definition(self):
        '''
        This function checks if all outputs are defined
        '''
        for node in self.nodes:
            if node.type == node_type.output_node:
                if node.gate == None:
                    raise FloatingOutput(node.name)
                
    def __repr__(self):
        '''
        This function returns the string representation of a circuit
        '''
        str_repr = "-------------------------------------------\n"
        str_repr = f"{str_repr}--------------Input Nodes------------------\n{self.input_list}\n"
        str_repr = f"{str_repr}-------------Output Nodes------------------\n{self.output_list}\n"
        str_repr = f"{str_repr}---------------Gate list-------------------\n"

        for node in self.nodes:
            if node.type != node_type.input_node:
                str_repr = f"{str_repr}{node.gate}\n"
        
        if self.levelized_nodes != None:
            str_repr = f"{str_repr}-------------Levelized circuit-------------\n"
            for level in self.levelized_nodes:
                str_repr = f"{str_repr}Level {level}: {self.levelized_nodes[level]}\n"

        return str_repr
    
    def levelize_circuit(self):
        '''
        This function levelizes the circuit and updates levelized_nodes
        '''
        if self.levelized_nodes != None:
            #circuit already levelized
            return
        
        self.levelized_nodes = {}
        
        for n in self.nodes:
            if n.level == None:
                n.find_level()

            if n.level in self.levelized_nodes:
                self.levelized_nodes[n.level].append(n)
            else:
                self.levelized_nodes[n.level] = [n]

        self.levelized_nodes = dict(sorted(self.levelized_nodes.items()))
        self.num_levels = len(self.levelized_nodes)

    def create_fault_list(self):
        '''
        This function creates the fault list for all the nodes in the circuit
        '''
        if self.__fault_list_created == True:
            return
        
        if self.levelized_nodes == None:
            raise CirNotLevelized()
        for level in self.levelized_nodes:
            for n in self.levelized_nodes[level]:
                n.create_fault_list()

        self.__fault_list_created = True

    def get_controllability(self):
        '''
        This function gets the controllability of each node in the circuit
        '''
        if self.levelized_nodes == None:
            raise CirNotLevelized()
        
        for level in self.levelized_nodes:
            for n in self.levelized_nodes[level]:
                n.get_controllability()
        
    def print_fault_list(self):
        '''
        This function prints the full fault list
        '''
        if not(self.__fault_list_created):
            print("Fault list not created")
            return
        
        total_faults = 0
        for level in self.levelized_nodes:
            for n in self.levelized_nodes[level]:
                print(f"Node: {n.name}, Faults: {n.fault_list}")
                total_faults += len(n.fault_list[0]) * 2

        print("Total number of faults: ", total_faults)

    def print_controllability(self):
        '''
        This function prints the controllability of each node in the circuit
        '''
        if self.levelized_nodes == None:
            raise CirNotLevelized()
        
        print("-------------Controllability of each node in the circuit-------------")
        for level in self.levelized_nodes:
            print(f"-------------Level {level}-------------")
            for n in self.levelized_nodes[level]:
                print(f"{n.name}: ({n.controllability.c0}, {n.controllability.c1})")

    def simulate(self, input_vector):
        '''
        This function simulates the circuit for the given input vector.
        The input vector should be a dictionary with the node names as the
        keys and values should be the node values
        '''

        for node_name in input_vector:
            self.nodes[self.node_index[node_name]].value = input_vector[node_name]
        
        for n in self.levelized_nodes[0]:
            if n.value == node_value.undefined:
                raise InputUndefined(n)
        
        for level in range(1, self.num_levels):
            for n in self.levelized_nodes[level]:
                n.gate.get_output()

    def select_fault(self, fault_string):
        fault_string_list = fault_string.split("-")
        input_node = None
        output_node = None
        fault_type = fault_types.sa0
        
        if len(fault_string_list) == 2:
            input_node = fault_string_list[0]
            fault_type = fault_string_list[1]
        else:
            input_node = fault_string_list[1]
            fault_type = fault_string_list[2] 
            output_node = fault_string_list[0]

        if input_node not in self.node_index:
            raise FaultReprError(fault_input= input_node)
        if fault_type not in ["0", "1"]:
            raise FaultReprError(fault_type = fault_type)
        if output_node != None:
            if output_node != "out":
                if output_node not in self.node_index:
                    raise FaultReprError(fault_output = output_node)
        
        if output_node != "out" and output_node != None:
            output_node = self.nodes[self.node_index[output_node]]
        
        if fault_type == "0":
            fault_type = fault_types.sa0
        else:
            fault_type = fault_types.sa1

        self.nodes[self.node_index[input_node]].select_fault(fault_type, output_node)

    def select_fault_from_user_input(self):
        print("Enter the fault to be selected in <output node name>-<input node name>-<0 or 1>  or <node name>-<0 or 1> format")
        print("To select the fault at the output, enter the fault format in \"out-<output node name>-<0 or 1>\"")
        fault_string = input("Enter fault string:")
        self.select_fault(fault_string)

    def __print_node_values_as_table(self, input_list, entries_per_row):
        '''
        prints node values as tables
        '''
        output_repr = {node_value.zero     : "0",
                       node_value.one      : "1",
                       node_value.d        : "D",
                       node_value.d_bar    : "D'",
                       node_value.undefined: "X"}
        
        index_row_1 = 0
        index_row_2 = 0
        for i in range(entries_per_row, len(input_list), entries_per_row):
            print("Node  ", end = "")
            while index_row_1 < i:
                print("| {:^5} ".format(input_list[index_row_1].name), end = "")
                index_row_1 += 1
            print("|")
            print("Value ", end = "")
            while index_row_2 < i:
                print("| {:^5} ".format(output_repr[input_list[index_row_2].value]), end = "")
                index_row_2 += 1
            print("|\n")


        if index_row_1 < len(input_list):
            print("Node  ", end = "")
            while index_row_1 < len(input_list):
                print("| {:^5} ".format(input_list[index_row_1].name), end = "")
                index_row_1 += 1
            print("|")
            print("Value ", end = "")
            while index_row_2 < len(input_list):
                print("| {:^5} ".format(output_repr[input_list[index_row_2].value]), end = "")
                index_row_2 += 1
            print("|")
        

    def print_node_values(self, input_entries_per_row = 5, internal_entries_per_row = 5, output_entries_per_row = 5):
        '''
        This function prints the values of all the nodes in the circuit. Output
        will be printed in the ascending order of levels 
        '''
        if (input_entries_per_row > 0):
            print(f'------Input Nodes------')
            self.__print_node_values_as_table(self.input_list, input_entries_per_row)

        if (internal_entries_per_row > 0):
            print(f'\n------Internal Nodes------')
            self.__print_node_values_as_table(self.internal_nodes, internal_entries_per_row)

        if (output_entries_per_row > 0):
            print(f'\n------Output Nodes------')
            self.__print_node_values_as_table(self.output_list, output_entries_per_row)


    def reset_circuit(self, reset_count = False):
        '''
        Resets all the circuit parameters
        '''
        for n in self.input_list:
            n.value = node_value.undefined
            n.selected_fault = None
            if reset_count:
                n.zero_count = 0
                n.one_count = 0

        for n in self.internal_nodes:
            n.value = node_value.undefined
            n.selected_fault = None
            if reset_count:
                n.zero_count = 0
                n.one_count = 0

        for n in self.output_list:
            n.value = node_value.undefined
            n.selected_fault = None
            if reset_count:
                n.zero_count = 0
                n.one_count = 0

    
    

In [15]:
def create_input_vector(num_inputs, choices = [node_value.zero, node_value.one]):
    
    total_input_vector = len(choices) ** num_inputs
    input_vectors = []
    for i in range(total_input_vector):
        for j in range(num_inputs):
            divisor = 1
            if j != 0:
                divisor = len(choices) ** j 
            value = [choices[(i // divisor) % len(choices)]]
            if j == 0:
                input_vectors.append(value)
            else:
                input_vectors[-1] = input_vectors[-1] + value

    return input_vectors

def int_to_binary_list(num, n):
    """
    Converts an integer to a binary representation in the form of a list of size n.

    Args:
        num (int): The integer to convert to binary.
        n (int): The size of the resulting list.
    
    Returns:
        list: A list of size n where each element is a binary digit (0 or 1).
    """
    if num >= 2 ** n:
        raise ValueError(f"The number {num} exceeds the maximum representable value for n = {n}.")

    # Convert the number to binary, remove the '0b' prefix, and pad with leading zeros
    binary_str = f"{num:0{n}b}"
    binary_list = [int(bit) for bit in binary_str]
    
    for i in range(len(binary_list)):
        if binary_list[i] == 0:
            binary_list[i] = node_value.zero
        else:
            binary_list[i] = node_value.one

    return binary_list

def monte_carlo_select(n):
    """
    Performs Monte Carlo simulations to select a number between 0 and 2^n - 1.

    Args:
        n (int): The number of bits, defining the range [0, 2^n - 1].
        num_simulations (int): Number of Monte Carlo simulations (default: 100000).
    
    Returns:
        int: A randomly selected number between 0 and 2^n - 1.
    """
    max_value = 2 ** n
    selected_number = random.randint(0, max_value - 1)
    return int_to_binary_list(selected_number, n)

def print_tables(dict_list, convert_node_value = True):
    '''
    This functions prints the list of dictionary in a table format
    '''
    headers = dict_list[0].keys()
    for h in headers:
        print("| {:<5} ".format(h), end = "")
    print("|")

    for d in dict_list:
        for h in d:
            if convert_node_value:
                print("| {:<5} ".format(node_value.str_repr[d[h]]), end = "")
            else:
                print("| {:<5} ".format(d[h]), end = "")
        print("|")


def test_circuit(circuit_under_test: circuit, iv_list, print_result = True):
    '''
    This function tests the given circuit with the input vector in the iv_list
    '''
    input_vector = {}
    
    for n in circuit_under_test.input_list:
        input_vector[n.name] = node_value.undefined

    results = []
    
    for iv in iv_list:
        i = 0
        for n in input_vector:
            input_vector[n] = iv[i]
            i = i + 1
        circuit_under_test.simulate(input_vector)
        results.append(copy.deepcopy(input_vector))
        for level in circuit_under_test.levelized_nodes:
            if level == 0:
                continue
            for n in circuit_under_test.levelized_nodes[level]:
                results[-1][n.name] = n.value
        circuit_under_test.reset_circuit()
    
    if (print_result):
        print_tables(results)


def print_scoap_simulation_comparison(circuit_under_test: circuit, print_by_level = False):
    '''
    This function prints each nodes controllability, number of zeros, number of ones and correspoding probabilites from each run
    '''
    dict_list = []
    if print_by_level:
        for level in circuit_under_test.levelized_nodes:
            data_dict = {"Level": level,
                         "PS(0)": 0,
                         "PS(1)": 0,
                         "PN(0)": 0,
                         "PN(1)": 0}
            for n in circuit_under_test.levelized_nodes[level]:
                data_dict["PS(0)"] += round((n.controllability.c0 / (n.controllability.c0 + n.controllability.c1)), 2)
                data_dict["PS(1)"] += round((n.controllability.c1 / (n.controllability.c0 + n.controllability.c1)), 2)
                data_dict["PN(0)"] += round((n.zero_count / (n.zero_count + n.one_count)), 2)
                data_dict["PN(1)"] += round((n.one_count / (n.zero_count + n.one_count)), 2)
            
            data_dict["PS(0)"] = round(data_dict["PS(0)"] / len(circuit_under_test.levelized_nodes[level]), 2)
            data_dict["PS(1)"] = round(data_dict["PS(1)"] / len(circuit_under_test.levelized_nodes[level]), 2)
            data_dict["PN(0)"] = round(data_dict["PN(0)"] / len(circuit_under_test.levelized_nodes[level]), 2)
            data_dict["PN(1)"] = round(data_dict["PN(1)"] / len(circuit_under_test.levelized_nodes[level]), 2)
            dict_list.append(data_dict)
    else:
        for level in circuit_under_test.levelized_nodes:
            for n in circuit_under_test.levelized_nodes[level]:
                data_dict = {"Node": n.name,
                            "C0": n.controllability.c0,
                            "C1": n.controllability.c1,
                            "N0": n.zero_count,
                            "N1": n.one_count,
                            "PS(0)": round((n.controllability.c0 / (n.controllability.c0 + n.controllability.c1)), 2),
                            "PS(1)": round((n.controllability.c1 / (n.controllability.c0 + n.controllability.c1)), 2),
                            "PN(0)": round((n.zero_count / (n.zero_count + n.one_count)), 2),
                            "PN(1)": round((n.one_count / (n.zero_count + n.one_count)), 2) }
                dict_list.append(data_dict)
    
    print("Note: PS(x): Probability of x from SCOAP analysis i.e PS(x) = (cx / (c0 + c1))\n      PN(x): Probability of x from Simulation i.e PN(x) = (nx / (n0 + n1))")
    print_tables(dict_list, False)

def perform_monte_carlo_test(circuit_under_test: circuit, num_tests):
    '''
    This test performs Monte carlo test on the circuit 
    '''
    iv_list = []
    iv_len = len(circuit_under_test.input_list)
    for n in range(0,num_tests):
        iv_list.append(monte_carlo_select(iv_len))
    
    test_circuit(circuit_under_test, iv_list, False)
        

    

# SCOAP analysis for p2.bench

In [16]:
circuit_under_test = circuit("p2.bench")
circuit_under_test.levelize_circuit()
print(circuit_under_test)
circuit_under_test.get_controllability()
circuit_under_test.print_controllability()


-------------------------------------------
--------------Input Nodes------------------
[a, b, c, d]
-------------Output Nodes------------------
[y, z]
---------------Gate list-------------------
2-input AND gate | Input nodes: a,b | Output node: g
2-input OR gate | Input nodes: g,c | Output node: k
1-input NOT gate | Input nodes: g | Output node: i
1-input NOT gate | Input nodes: b | Output node: e
2-input NAND gate | Input nodes: i,k | Output node: w
3-input NOR gate | Input nodes: k,e,d | Output node: x
1-input NOT gate | Input nodes: w | Output node: y
1-input NOT gate | Input nodes: x | Output node: z
-------------Levelized circuit-------------
Level 0: [a, b, c, d]
Level 1: [g, e]
Level 2: [k, i]
Level 3: [w, x]
Level 4: [y, z]

-------------Controllability of each node in the circuit-------------
-------------Level 0-------------
a: (1, 1)
b: (1, 1)
c: (1, 1)
d: (1, 1)
-------------Level 1-------------
g: (2, 3)
e: (2, 2)
-------------Level 2-------------
k: (4, 2)
i: (4, 3)
---

### Comparison of SCOAP probablities and simulation results of p2.bench for all 16 input combinations

In [17]:
iv_list = create_input_vector(len(circuit_under_test.input_list))
circuit_under_test.reset_circuit(True)
test_circuit(circuit_under_test, iv_list)

| a     | b     | c     | d     | g     | e     | k     | i     | w     | x     | y     | z     |
| 0     | 0     | 0     | 0     | 0     | 1     | 0     | 1     | 1     | 0     | 0     | 1     |
| 1     | 0     | 0     | 0     | 0     | 1     | 0     | 1     | 1     | 0     | 0     | 1     |
| 0     | 1     | 0     | 0     | 0     | 0     | 0     | 1     | 1     | 1     | 0     | 0     |
| 1     | 1     | 0     | 0     | 1     | 0     | 1     | 0     | 1     | 0     | 0     | 1     |
| 0     | 0     | 1     | 0     | 0     | 1     | 1     | 1     | 0     | 0     | 1     | 1     |
| 1     | 0     | 1     | 0     | 0     | 1     | 1     | 1     | 0     | 0     | 1     | 1     |
| 0     | 1     | 1     | 0     | 0     | 0     | 1     | 1     | 0     | 0     | 1     | 1     |
| 1     | 1     | 1     | 0     | 1     | 0     | 1     | 0     | 1     | 0     | 0     | 1     |
| 0     | 0     | 0     | 1     | 0     | 1     | 0     | 1     | 1     | 0     | 0     | 1     |
| 1     | 0     | 0 

In [18]:
print_scoap_simulation_comparison(circuit_under_test)

Note: PS(x): Probability of x from SCOAP analysis i.e PS(x) = (cx / (c0 + c1))
      PN(x): Probability of x from Simulation i.e PN(x) = (nx / (n0 + n1))
| Node  | C0    | C1    | N0    | N1    | PS(0) | PS(1) | PN(0) | PN(1) |
| a     | 1     | 1     | 8     | 8     | 0.5   | 0.5   | 0.5   | 0.5   |
| b     | 1     | 1     | 8     | 8     | 0.5   | 0.5   | 0.5   | 0.5   |
| c     | 1     | 1     | 8     | 8     | 0.5   | 0.5   | 0.5   | 0.5   |
| d     | 1     | 1     | 8     | 8     | 0.5   | 0.5   | 0.5   | 0.5   |
| g     | 2     | 3     | 12    | 4     | 0.4   | 0.6   | 0.75  | 0.25  |
| e     | 2     | 2     | 8     | 8     | 0.5   | 0.5   | 0.5   | 0.5   |
| k     | 4     | 2     | 6     | 10    | 0.67  | 0.33  | 0.38  | 0.62  |
| i     | 4     | 3     | 4     | 12    | 0.57  | 0.43  | 0.25  | 0.75  |
| w     | 6     | 5     | 6     | 10    | 0.55  | 0.45  | 0.38  | 0.62  |
| x     | 2     | 8     | 15    | 1     | 0.2   | 0.8   | 0.94  | 0.06  |
| y     | 6     | 7     | 10    

### Comparison of Scoap analysis and Monte Carlo simulation for p2.bench

In [19]:
circuit_under_test.reset_circuit(True)
perform_monte_carlo_test(circuit_under_test, 1000)
print_scoap_simulation_comparison(circuit_under_test)

Note: PS(x): Probability of x from SCOAP analysis i.e PS(x) = (cx / (c0 + c1))
      PN(x): Probability of x from Simulation i.e PN(x) = (nx / (n0 + n1))
| Node  | C0    | C1    | N0    | N1    | PS(0) | PS(1) | PN(0) | PN(1) |
| a     | 1     | 1     | 505   | 495   | 0.5   | 0.5   | 0.51  | 0.49  |
| b     | 1     | 1     | 481   | 519   | 0.5   | 0.5   | 0.48  | 0.52  |
| c     | 1     | 1     | 499   | 501   | 0.5   | 0.5   | 0.5   | 0.5   |
| d     | 1     | 1     | 489   | 511   | 0.5   | 0.5   | 0.49  | 0.51  |
| g     | 2     | 3     | 750   | 250   | 0.4   | 0.6   | 0.75  | 0.25  |
| e     | 2     | 2     | 519   | 481   | 0.5   | 0.5   | 0.52  | 0.48  |
| k     | 4     | 2     | 365   | 635   | 0.67  | 0.33  | 0.36  | 0.64  |
| i     | 4     | 3     | 250   | 750   | 0.57  | 0.43  | 0.25  | 0.75  |
| w     | 6     | 5     | 385   | 615   | 0.55  | 0.45  | 0.39  | 0.61  |
| x     | 2     | 8     | 941   | 59    | 0.2   | 0.8   | 0.94  | 0.06  |
| y     | 6     | 7     | 615   

### Comparison of Monte carlo simulation and SCOAP analysis for c432.bench

In [20]:
circuit_under_test = circuit("c432.bench")
circuit_under_test.levelize_circuit()
circuit_under_test.get_controllability()
circuit_under_test.print_controllability()

-------------Controllability of each node in the circuit-------------
-------------Level 0-------------
1: (1, 1)
4: (1, 1)
8: (1, 1)
11: (1, 1)
14: (1, 1)
17: (1, 1)
21: (1, 1)
24: (1, 1)
27: (1, 1)
30: (1, 1)
34: (1, 1)
37: (1, 1)
40: (1, 1)
43: (1, 1)
47: (1, 1)
50: (1, 1)
53: (1, 1)
56: (1, 1)
60: (1, 1)
63: (1, 1)
66: (1, 1)
69: (1, 1)
73: (1, 1)
76: (1, 1)
79: (1, 1)
82: (1, 1)
86: (1, 1)
89: (1, 1)
92: (1, 1)
95: (1, 1)
99: (1, 1)
102: (1, 1)
105: (1, 1)
108: (1, 1)
112: (1, 1)
115: (1, 1)
-------------Level 1-------------
118: (2, 2)
119: (2, 2)
122: (2, 2)
123: (2, 2)
126: (2, 2)
127: (2, 2)
130: (2, 2)
131: (2, 2)
134: (2, 2)
135: (2, 2)
138: (2, 2)
139: (2, 2)
142: (2, 2)
143: (2, 2)
146: (2, 2)
147: (2, 2)
150: (2, 2)
151: (2, 2)
-------------Level 2-------------
154: (4, 2)
157: (2, 4)
158: (2, 4)
159: (4, 2)
162: (4, 2)
165: (4, 2)
168: (4, 2)
171: (4, 2)
174: (4, 2)
177: (4, 2)
180: (4, 2)
183: (2, 4)
184: (2, 4)
185: (2, 4)
186: (2, 4)
187: (2, 4)
188: (2, 4)
189: (2, 4

In [ ]:
circuit_under_test.reset_circuit(True)
perform_monte_carlo_test(circuit_under_test, 1000)
print_scoap_simulation_comparison(circuit_under_test, True)

Note: PS(x): Probability of x from SCOAP analysis i.e PS(x) = (cx / (c0 + c1))
      PN(x): Probability of x from Simulation i.e PN(x) = (nx / (n0 + n1))
| Level | PS(0) | PS(1) | PN(0) | PN(1) |
| 0     | 0.5   | 0.5   | 0.49  | 0.51  |
| 1     | 0.5   | 0.5   | 0.51  | 0.49  |
| 2     | 0.44  | 0.56  | 0.59  | 0.41  |
| 3     | 0.21  | 0.79  | 0.96  | 0.04  |
| 4     | 0.77  | 0.23  | 0.04  | 0.96  |
| 5     | 0.62  | 0.38  | 0.59  | 0.41  |
| 6     | 0.83  | 0.17  | 0.13  | 0.87  |
| 7     | 0.22  | 0.78  | 0.86  | 0.14  |
| 8     | 0.63  | 0.37  | 0.27  | 0.73  |
| 9     | 0.64  | 0.36  | 0.46  | 0.54  |
| 10    | 0.91  | 0.09  | 0.09  | 0.91  |
| 11    | 0.52  | 0.48  | 0.61  | 0.39  |
| 12    | 0.48  | 0.52  | 0.39  | 0.61  |
| 13    | 0.96  | 0.04  | 0.29  | 0.71  |
| 14    | 0.8   | 0.2   | 0.15  | 0.85  |
| 15    | 0.27  | 0.73  | 0.87  | 0.13  |
| 16    | 0.71  | 0.29  | 0.12  | 0.88  |
| 17    | 0.59  | 0.41  | 0.51  | 0.49  |
